In [15]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import glob

from dipy.core.gradients import gradient_table
from dipy.core.ndindex import ndindex
from dipy.core.sphere import faces_from_sphere_vertices, unique_edges
from dipy.data import get_sphere, HemiSphere, Sphere
from dipy.direction.peaks import peak_directions
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.io.streamline import load_trk
from dipy.io.streamline import save_trk
from dipy.reconst.gqi import GeneralizedQSamplingModel
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.reconst.shm import sf_to_sh, sph_harm_lookup, smooth_pinv
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking,CmcTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask
from dipy.viz import window, actor, fvtk

In [3]:
os.chdir('/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001/')
wd = ('/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001/')

In [4]:
sphere642 = get_sphere('symmetric642')
sphere642.vertices = sphere642.vertices.astype('float')
sphere642.edges = sphere642.edges.astype('uint16')
sphere642.faces = sphere642.faces.astype('uint16')
sphere642.phi = sphere642.phi.astype('float')

In [ ]:
#parameters
dl = 1.4
#setting variables
for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    bvals, bvecs = read_bvals_bvecs(os.path.join(sub, 'all_shells/bvals'), os.path.join(sub, 'all_shells/bvecs'))
    img = nib.load(os.path.join(sub, 'all_shells/data_corr.nii.gz'))
    data = img.get_data()
    gtab = gradient_table(bvals,bvecs)
    print('Computing matrix A for'+str(sub))
    #computing matrix A
    l_values = np.sqrt(gtab.bvals * 0.01506)
    tmp=np.tile(l_values, (3,1))
    gradsT = gtab.bvecs.T
    b_vector = gradsT * tmp
    b_vector = b_vector.T
    gqi_vector = np.real(np.sinc(np.dot(b_vector, sphere642.vertices.T)* dl/np.pi))
    print('Computing ODFs for'+str(sub))
    odfs = np.dot(data, gqi_vector)
    print('Saving ODFs for'+str(sub))
    #save odfs 
    nib.save(nib.Nifti1Image(odfs.astype(np.float32),
                                  img.affine, img.header), os.path.join(sub, 'all_shells/odfs.nii.gz'))

In [ ]:
%%script tcsh
foreach sub (mgh_????)
        echo $sub
        set input = $sub/all_shells/odfs.nii.gz
        set output = $sub/all_shells/odfs_masked.nii.gz
        set mask = $sub/nodif_brain_mask.nii.gz
        fslmaths
        echo 'processing' $input 
        fslmaths $input -mas $mask $output
end

In [ ]:
#min/max normalization
for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    print('Loading ODFs for'+str(sub))
    mask = nib.load(os.path.join(sub, 'nodif_brain_mask.nii.gz')).get_data()
    img = nib.load(os.path.join(sub, 'all_shells/odfs_masked.nii.gz'))
    odfs = img.get_data()
    print('Normalizing ODFs for'+str(sub))
    ijk = np.ascontiguousarray(np.array(np.nonzero(mask)).T)
    shape = odfs.shape[:-1]
    odfs_norm = np.zeros ((shape + (len(sphere642.vertices),)))
    for (k, center) in enumerate(ijk):
        m = odfs[tuple(center.astype(np.int))].copy()
        m = m - (np.abs(m).min())
        m = m / (np.abs(m).max())
        odfs_norm[tuple(center.astype(np.int))] = m
    print('Saving Normalized ODFs for'+str(sub))
    nib.save(nib.Nifti1Image(odfs_norm.astype(np.float32),
                                  img.affine, img.header), os.path.join(sub, 'all_shells/odfs_norm.nii.gz'))

In [ ]:
#Extract peaks values and directions
#set maximum number of peaks relatve peak threshold and min separation angle
npeaks=3
relative_peak_threshold=.5
min_separation_angle=25

for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    #load files
    mask = nib.load(os.path.join(sub, 'nodif_brain_mask.nii.gz')).get_data()
    img = nib.load(os.path.join(sub, 'all_shells/odfs_norm.nii.gz'))
    odfs = img.get_data()
    shape=mask.shape    
    peak_dirs = np.zeros ((shape + (npeaks, 3)))
    peak_values = np.zeros ((shape + (npeaks,)))
    peak_indices = np.zeros((shape + (npeaks,)), dtype='int')
    odfs =odfs.astype(float)
    for idx in ndindex(shape):
        if not mask[idx]:
            continue
            direction, pk, indices = peak_directions(odfs[idx], sphere642, 
                                                     relative_peak_threshold, min_separation_angle)
            n = min(npeaks, pk.shape[0])
            peak_dirs[idx][:n] = direction[:n]
            peak_values[idx][:n] = pk[:n]
            peak_indices[idx][:n] = indices[:n]
            #save peaks directions
#     nib.save(nib.Nifti1Image(reshape_peaks_for_visualization(peak_dirs),
#                                      img.affine),os.path.join(sub, 'all_shells/peaks.nii.gz'))

In [ ]:
#checking ODFs
interactive = False

for sub in glob.glob(os.path.join(wd, 'mgh_*')):
    print('Loading ODFs for'+str(sub))
    #load files 
    img = nib.load(os.path.join(sub, 'all_shells/odfs_norm.nii.gz'))
    odfs = img.get_data()
    #actors
    r = window.Renderer()
    gqi_odfs_actor = actor.odf_slicer(odfs, sphere=sphere642, scale=0.6, norm=True, colormap= 'jet')
#     gqi_peaks_actor = actor.peak_slicer(peak_dirs, peak_values, colors=(1,0,0))
    gqi_odfs_actor.GetProperty().SetOpacity(0.6)
    gqi_odfs_actor.display_extent(0, 140, 90, 90, 0, 96)
#     gqi_peaks_actor.display_extent(83, 83, 69, 69, 59, 59)
    r.set_camera(position=(0, 10, 0))
    r.add(gqi_odfs_actor)
#     r.add(gqi_peaks_actor)
    print('Saving illustration as gqi_odfs.png')
    if interactive:
        window.show(r)
    else:
        window.record(r, out_path=os.path.join(sub, 'all_shells/odfs.png'), size=(900, 900))
    r = None
    img = None
    odfs = None
    gqi_odfs_actor = None
    
    

Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1023


/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/viz/actor.py:821: RuntimeWarning: invalid value encountered in true_divide
  m /= np.abs(m).max()
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/matplotlib/colors.py:504: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1015
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1001
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1018
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1002
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1010
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1027
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1008
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1019
Saving illustration as gqi_odfs.png
Loading ODFs for/space/hemera/1/users/cmaffei/hcp_processing/mgh_1033
